<a href="https://colab.research.google.com/github/yeremiapane/Project-Capstone/blob/master/capstone_rumah123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
from google.colab import drive

drive.mount('/content/drive')

# data harga rumah di Jakarta, Bekasi, dan Depok dari sumber rumah123.com
df_jkt_r123 = pd.read_csv("/content/drive/MyDrive/data-collecting/01jakarta-rumah123.csv", header=0)
df_bks_r123 = pd.read_csv("/content/drive/MyDrive/data-collecting/05bekasi-rumah123.csv", header=0)
df_dpk_r123 = pd.read_csv("/content/drive/MyDrive/data-collecting/03depok-rumah123.csv", header=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_jkt_r123

,ui-organisms-card-r123-featured__middle-section__price,ui-organisms-card-r123-featured__middle-section__address,attribute-text,attribute-text 2,attribute-text 3,attribute-value,attribute-value 2
0,"Rp 2,7 Miliar","Jelambar, Jakarta Barat",4.0,4.0,1.0,92 m²,218 m²
1,"Rp 2,7 Miliar","Jelambar, Jakarta Barat",4.0,3.0,NaN,92 m²,231 m²
2,Rp 35 Miliar,"Kemayoran, Jakarta Pusat",5.0,11.0,7.0,746 m²,1100 m²
3,"Rp 8,5 Miliar","Pantai Indah Kapuk, Jakarta Utara",4.0,3.0,2.0,300 m²,350 m²
4,"Rp 16,5 Miliar","Permata Buana, Jakarta Barat",3.0,3.0,1.0,300 m²,450 m²
...,...,...,...,...,...,...,...
1975,"Rp 10,5 Miliar","Kepa Duri, Jakarta Barat",5.0,8.0,1.0,234 m²,500 m²
1976,"Rp 15,5 Miliar","Pancoran, Jakarta Selatan",6.0,4.0,NaN,522 m²,450 m²
1977,"Rp 5,25 Miliar","Tomang, Jakarta Barat",4.0,2.0,NaN,228 m²,331 m²
1978,"Rp 4,5 Miliar","Cengkareng, Jakarta Barat",4.0,4.0,NaN,180 m²,135 m²


In [4]:
''' data rumah Jakarta, Bekasi, Depok dari sumber rumah123.com'''
# menghilangkan kolom yang tidak perlu
df_jkt_r123.drop(['attribute-text 3','attribute-value'],axis=1,inplace=True)
df_bks_r123.drop(['attribute-text 3','attribute-value'],axis=1,inplace=True)
df_dpk_r123.drop(['attribute-text 3','attribute-value'],axis=1,inplace=True)

# menghapus missing values
df_jkt_r123.dropna(axis=0,inplace=True)
df_bks_r123.dropna(axis=0,inplace=True)
df_dpk_r123.dropna(axis=0,inplace=True)

# rename kolom
df_jkt_r123 = df_jkt_r123.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "price", "ui-organisms-card-r123-featured__middle-section__address": "location", "attribute-text": "bed", "attribute-text 2": "bath", "attribute-value 2": "area"})
df_bks_r123 = df_bks_r123.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "price", "ui-organisms-card-r123-featured__middle-section__address": "location", "attribute-text": "bed", "attribute-text 2": "bath", "attribute-value 2": "area"})
df_dpk_r123 = df_dpk_r123.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "price", "ui-organisms-card-r123-featured__middle-section__address": "location", "attribute-text": "bed", "attribute-text 2": "bath", "attribute-value 2": "area"})

# ubah value area menjadi data numerik
df_jkt_r123['area'] = df_jkt_r123.area.str.extract('(\d+)')
df_bks_r123['area'] = df_bks_r123.area.str.extract('(\d+)')
df_dpk_r123['area'] = df_dpk_r123.area.str.extract('(\d+)')

# menyamakan value location
df_jkt_r123['location'] = "Jakarta"
df_bks_r123['location'] = "Bekasi"
df_dpk_r123['location'] = "Depok"

# hilangkan tulisan 'Rp'
df_jkt_r123['price'] = df_jkt_r123.price.str.strip('Rp')
df_bks_r123['price'] = df_bks_r123.price.str.strip('Rp')
df_dpk_r123['price'] = df_dpk_r123.price.str.strip('Rp')

# reorder kolom
df_jkt_r123 = df_jkt_r123.reindex(columns=['area', 'bed', 'bath', 'location', 'price'])
df_bks_r123 = df_bks_r123.reindex(columns=['area', 'bed', 'bath', 'location', 'price'])
df_dpk_r123 = df_dpk_r123.reindex(columns=['area', 'bed', 'bath', 'location', 'price'])

In [5]:
# fungsi ubah string ke numerik
def converter(x):
  if 'juta' in x:
      return f"{(float(x.strip(' juta'))*1000000):,.2f}"
  elif 'Miliar' in x:
      return f"{(float(x.strip(' Miliar'))*1000000000):,.2f}"

# transformasi value price
df_jkt_r123['p0'] = df_jkt_r123.price.str.split("-", n=1, expand=True)[0]
df_jkt_r123['p0'] = df_jkt_r123['p0'].str.replace('.', '')
df_jkt_r123['p0'] = df_jkt_r123['p0'].str.replace(',', '.').apply(converter)

df_bks_r123['p0'] = df_bks_r123.price.str.split("-", n=1, expand=True)[0]
df_bks_r123['p0'] = df_bks_r123['p0'].str.replace('.', '')
df_bks_r123['p0'] = df_bks_r123['p0'].str.replace(',', '.').apply(converter)

df_dpk_r123['p0'] = df_dpk_r123.price.str.split("-", n=1, expand=True)[0]
df_dpk_r123['p0'] = df_dpk_r123['p0'].str.replace('.', '')
df_dpk_r123['p0'] = df_dpk_r123['p0'].str.replace(',', '.').apply(converter)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [6]:
# ganti value price dengan p0
df_jkt_r123 = df_jkt_r123.drop(columns='price')
df_jkt_r123 = df_jkt_r123.rename(columns={"p0": "price"})

df_bks_r123 = df_bks_r123.drop(columns='price')
df_bks_r123 = df_bks_r123.rename(columns={"p0": "price"})

df_dpk_r123 = df_dpk_r123.drop(columns='price')
df_dpk_r123 = df_dpk_r123.rename(columns={"p0": "price"})


# hilangkan koma
df_jkt_r123['price'] = df_jkt_r123['price'].str.replace(',', '')
df_bks_r123['price'] = df_bks_r123['price'].str.replace(',', '')
df_dpk_r123['price'] = df_dpk_r123['price'].str.replace(',', '')

In [7]:
# ubah tipe data
df_jkt_r123 = df_jkt_r123.astype({"area":'float', "bed":'int', "bath": 'int', "price": 'float'}) 
df_bks_r123 = df_bks_r123.astype({"area":'float', "bed":'int', "bath": 'int', "price": 'float'}) 
df_dpk_r123 = df_dpk_r123.astype({"area":'float', "bed":'int', "bath": 'int', "price": 'float'}) 


In [8]:
# menangani missing value
df_jkt_r123.isnull().sum()
df_jkt_r123['price'].fillna(df_jkt_r123['price'].mean(), inplace=True)

df_bks_r123.isnull().sum()
df_bks_r123['price'].fillna(df_bks_r123['price'].mean(), inplace=True)

df_dpk_r123.isnull().sum()
df_dpk_r123['price'].fillna(df_dpk_r123['price'].mean(), inplace=True)


# ganti tipe data price menjadi int
df_jkt_r123['price'] = df_jkt_r123.price.astype('int')
df_bks_r123['price'] = df_bks_r123.price.astype('int')
df_dpk_r123['price'] = df_dpk_r123.price.astype('int')


In [9]:
df_jkt_r123

,area,bed,bath,location,price
0,218.0,4,4,Jakarta,2700000000
1,231.0,4,3,Jakarta,2700000000
2,1100.0,5,11,Jakarta,35000000000
3,350.0,4,3,Jakarta,8500000000
4,450.0,3,3,Jakarta,16500000000
...,...,...,...,...,...
1975,500.0,5,8,Jakarta,10500000000
1976,450.0,6,4,Jakarta,15500000000
1977,331.0,4,2,Jakarta,5250000000
1978,135.0,4,4,Jakarta,4500000000


In [10]:
df_jkt_r123.dtypes

area        float64
bed           int64
bath          int64
location     object
price         int64
dtype: object

In [11]:
# gabung ketiga dataframe menjadi 1
df_r123 = pd.concat([df_jkt_r123, df_bks_r123, df_dpk_r123])
df_r123

,area,bed,bath,location,price
0,218.0,4,4,Jakarta,2700000000
1,231.0,4,3,Jakarta,2700000000
2,1100.0,5,11,Jakarta,35000000000
3,350.0,4,3,Jakarta,8500000000
4,450.0,3,3,Jakarta,16500000000
...,...,...,...,...,...
1939,80.0,3,2,Depok,1820469858
1940,127.0,3,2,Depok,1500000000
1941,138.0,3,3,Depok,1350000000
1942,35.0,2,1,Depok,1820469858


In [13]:
df_r123.to_csv('data_rumah123.csv')
!cp data_rumah123.csv "/content/drive/MyDrive/data-collecting"